<a href="https://colab.research.google.com/github/charleslien/advent-of-code/blob/main/2024/day17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/charleslien/advent-of-code aoc
from aoc.lib import *

Cloning into 'aoc'...
remote: Enumerating objects: 440, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 440 (delta 127), reused 187 (delta 97), pack-reused 196 (from 1)
Receiving objects: 100% (440/440), 560.41 KiB | 1.86 MiB/s, done.
Resolving deltas: 100% (172/172), done.


In [2]:
raw = '''Register A: 64012472
Register B: 0
Register C: 0

Program: 2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0'''

In [3]:
blocks = process_raw(raw)

Splitting into blocks...
2 blocks found.
The blocks have the following number of lines:
  3
  1


In [4]:
p = re.compile(r'Register [ABC]: (\d+)')
regs = list(map(int, (p.fullmatch(line).group(1) for line in blocks[0])))
prog = list(map(int, blocks[1][0].split()[1].split(',')))
ip = 0

def combo(operand):
  if operand > 3:
    return regs[operand - 4]
  return operand
def operation(opcode, operand):
  global regs, ip
  if opcode == 0:
    regs[0] = int(regs[0] / (1 << combo(operand)))
  elif opcode == 1:
    regs[1] ^= operand
  elif opcode == 2:
    regs[1] = combo(operand) % 8
  elif opcode == 3:
    if regs[0] != 0:
      ip = operand - 2
  elif opcode == 4:
    regs[1] ^= regs[2]
  elif opcode == 5:
    return combo(operand) % 8
  elif opcode == 6:
    regs[1] = regs[0] // (1 << combo(operand))
  elif opcode == 7:
    regs[2] = regs[0] // (1 << combo(operand))

def run(prog):
  global ip
  output = []
  ip = 0
  while ip < len(prog):
    if (val := operation(prog[ip], prog[ip + 1])) is not None:
      output.append(val)
    ip += 2
  return output
','.join(map(str, run(prog)))

'1,0,2,0,5,7,2,1,3'

In [5]:
def combo_d(operand):
  if operand == 7:
    return ''
  if operand > 3:
    return regs[operand - 4]
  return str(operand)
def operation_d(opcode, operand):
  global regs, ip
  c = combo_d(operand)
  if opcode == 0:
    regs[0] = f'({regs[0]} >> {c})'
  elif opcode == 1:
    regs[1] = f'({regs[1]} ^ {operand})'
  elif opcode == 2:
    regs[1] = f'({c} % 8)'
  elif opcode == 3:
    return f'jnz, ip={operand}, {regs[0]} != 0'
  elif opcode == 4:
    regs[1] = f'({regs[1]} ^ {regs[2]})'
  elif opcode == 5:
    return f'{c} % 8'
  elif opcode == 6:
    regs[1] = f'({regs[0]} >> {c})'
  elif opcode == 7:
    regs[2] = f'({regs[0]} >> {c})'

def print_description(prog):
  global regs
  regs = ['a', '0', '0']
  ip = 0
  while ip < len(prog):
    if (val := operation_d(prog[ip], prog[ip + 1])) is not None:
      print(val)
    ip += 2
print_description(prog)
print(regs)

((((a % 8) ^ 7) ^ 7) ^ (a >> ((a % 8) ^ 7))) % 8
jnz, ip=0, (a >> 3) != 0
['(a >> 3)', '((((a % 8) ^ 7) ^ 7) ^ (a >> ((a % 8) ^ 7)))', '(a >> ((a % 8) ^ 7))']


In [6]:
# Note: always checks 3 bits at a time
stack = [0]
while stack:
  a = stack.pop()
  for d in range(8):
    if not (a + d):
      continue
    regs = [a * 8 + d, 0, 0]
    result = run(prog)
    if result[0] == prog[-len(result)]:
      if len(result) == len(prog):
        print(a * 8 + d)
        break
      stack.append(a * 8 + d)

265652340990875
